<a href="https://colab.research.google.com/github/mehrdadrashidian/CG-Ecosystem/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import kagglehub

# تنظیم مسیرها
img_height = 150
img_width = 150
batch_size = 32

# فرض بر این است که داده‌ها در همین مسیر هستند
# Download latest version
path = kagglehub.dataset_download("alessiocorrado99/animals10")
images_path = path
raw_img_path = os.path.join(images_path, 'raw-img')

# تعریف شی ImageDataGenerator برای پردازش تصاویر
datagen = ImageDataGenerator(rescale=1.0/255.0)  # نرمال‌سازی تصاویر

# بارگذاری داده‌های آموزشی
train_data = datagen.flow_from_directory(
    raw_img_path,  # مسیر پوشه raw-img
    target_size=(img_height, img_width),  # اندازه تصاویر
    batch_size=batch_size,  # تعداد نمونه‌ها در هر بچ
    class_mode='categorical',  # نوع برچسب‌ها (چندکلاسه)
    shuffle=True  # مخلوط کردن داده‌ها
)

# بارگذاری داده‌های اعتبارسنجی
val_data = datagen.flow_from_directory(
    raw_img_path,  # مسیر پوشه raw-img
    target_size=(img_height, img_width),  # اندازه تصاویر
    batch_size=batch_size,  # تعداد نمونه‌ها در هر بچ
    class_mode='categorical',  # نوع برچسب‌ها (چندکلاسه)
    shuffle=False  # عدم مخلوط کردن داده‌ها برای اعتبارسنجی
)

# بارگذاری مدل VGG16 از پیش آموزش‌داده‌شده بدون لایه‌های نهایی
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# فریز کردن لایه‌های مدل پیش‌آموزش‌داده‌شده
for layer in base_model.layers:
    layer.trainable = False

# افزودن لایه‌های جدید برای مسئله شما (تشخیص حیوانات)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # تبدیل به وکتور 1D
x = layers.Dense(256, activation='relu')(x)  # لایه کاملاً متصل
x = layers.Dropout(0.5)(x)  # برای جلوگیری از overfitting
predictions = layers.Dense(len(train_data.class_indices), activation='softmax')(x)  # تعداد کلاس‌های شما

# ساخت مدل نهایی
model = models.Model(inputs=base_model.input, outputs=predictions)

# کامپایل مدل
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# نمایش ساختار مدل
model.summary()

# آموزش مدل جدید با داده‌های شما
epochs = 10
history = model.fit(
    train_data,  # داده‌های آموزشی
    validation_data=val_data,  # داده‌های اعتبارسنجی
    epochs=epochs  # تعداد دوره‌های آموزشی
)

# ذخیره مدل آموزش‌داده‌شده در فرمت h5
model.save('animal_classifier_transfer_learning.h5')

print("Model saved as 'animal_classifier_transfer_learning.h5'")


100%|██████████| 586M/586M [00:05<00:00, 119MB/s]

Extracting files...


Found 26179 images belonging to 10 classes.
Found 26179 images belonging to 10 classes.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 150, 150, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 150, 150, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 75, 75, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 75, 75, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 37, 37, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 18, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 18, 18, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 9, 9, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,3

...

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


819/819 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5378 - loss: 1.3707